# Shors Algorithm

## Task 1: A classical implementation

First, we need to check special cases for Shor's Algorithm

### 1.1 IsEven

In [ ]:
%kata IsEven_Test

operation IsEven(N : Int) : Bool {
    return (N%2==0);
}

### 1.2 IsPrime

In [ ]:
%kata IsPrime_Test

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

operation IsPrime(N : Int) : Bool {
    if (N <= 3) {
        return (N > 1);
    }
    if (N%2 == 0) or (N%3 == 0) {
        return false;
    }
    let sqrtN = Truncate(Sqrt(IntAsDouble(N)));
    for i in 5..6..sqrtN {
        if (N%i == 0) or (N%(i+2) == 0) {
            return false;
        }

    }
    return true;
}

GCD operation

In [1]:
%kata GreatestCommonDivisor_Test

open Microsoft.Quantum.Math;
operation GreatestCommonDivisor (a : Int, N : Int) : Int {
    let min = Min([a,N]);
    mutable gcd = 1;
    for i in 2..min {
        if (a%i == 0 and N%i == 0) {
            set gcd = i;
        }
    }
    return gcd;
}

Success!

With these two tests, we can confirm that N is a product of two distinct prime numbers, so $N = pq$.
// Explain order finding

### 1.3 Classical order finding

In [ ]:
%kata FindOrderClassical_Test

operation FindOrderClassical(a : BigInt, N: BigInt) : Int {
    mutable power = 0;
    repeat {
        set power += 1;
    } until (a^power % N == 1L);
    return power;
}

FindOrderClassical

In [4]:
open Microsoft.Quantum.Convert;
operation FindOrderTest() : Unit {
    for a in 2..5 {
        for N in 10..15 {
            if GreatestCommonDivisor(a, N) == 1 {
                Message(IntAsString(a) + ", " + IntAsString(N));
                Message(IntAsString(FindOrderClassical(IntAsBigInt(a),IntAsBigInt(N))));
            }
        }
    }
}

FindOrderTest

In [5]:
%simulate FindOrderTest

2, 11
10
2, 13
12
2, 15
4
3, 10
4
3, 11
5
3, 13
3
3, 14
6
4, 11
5
4, 13
6
4, 15
2
5, 11
5
5, 12
2
5, 13
4
5, 14
6


()

In [ ]:
%kata Test_Test

open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;

operation Test(qs : Qubit[]) : Unit {
    ApplyToEach(H,qs);
}

In [ ]:
%kata OrderFindingOracle_Test

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Math;

operation OrderFindingOracle (a : Int, N : Int, power : Int, target : Qubit[]) : Unit is Adj {
    MultiplyByModularInteger(ExpModI(a, power, N), N, LittleEndian(target));
}

In [ ]:
operation OrderFindingTest () : Unit {
    let (a, N, r) = (2, 15, 3);
    use register = Qubit[BitSizeI(N)];
    ApplyToEachCA(H,register);
    OrderFindingOracle(a, N, r, register);
}

In [ ]:
%simulate OrderFindingTest